In [2]:
import dask.dataframe as dd
from dask.distributed import Client
import numpy as np
import pandas as pd
import gc


In [3]:
client = Client()
client

/media/ubuntu/ElementsSE/2022_finvcup_baseline-master/env/bisai/lib/python3.7/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44547 instead
  f"Port {expected} is already in use.\n"
distributed.diskutils - INFO - Found stale lock file and directory '/media/local_disk/jieli/forta/Algorithm/State transition graph embedding/dask-worker-space/worker-b2zk4mzy', purging
distributed.diskutils - INFO - Found stale lock file and directory '/media/local_disk/jieli/forta/Algorithm/State transition graph embedding/dask-worker-space/worker-cgga6fvd', purging
distributed.diskutils - INFO - Found stale lock file and directory '/media/local_disk/jieli/forta/Algorithm/State transition graph embedding/dask-worker-space/worker-lbg2gyqm', purging
distributed.diskutils - INFO - Found stale lock file and directory '/media/local_disk/jieli/forta/Algorithm/State transition graph embedding/dask-worker-s

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:44547/status,
Dashboard: http://127.0.0.1:44547/status,Workers: 7
Total threads: 28,Total memory: 125.50 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44619,Workers: 7
Dashboard: http://127.0.0.1:44547/status,Total threads: 28
Started: Just now,Total memory: 125.50 GiB
Comm: tcp://127.0.0.1:41309,Total threads: 4
Dashboard: http://127.0.0.1:42689/status,Memory: 17.93 GiB
Nanny: tcp://127.0.0.1:40601,


期望得到的信息: block_number, timestamp, transaction_hash, from_address, to_address, edge_type, value, token_id, token_address, gas, gas_price

In [4]:
# EOA transaction

data_dir = 'kaggle_data/forta-protect-web3'

TX_DTYPES = {'value': 'object', 'gas_price': 'object', 'receipt_status': 'object', 'receipt_contract_address': 'object', 'receipt_contract_address': 'object', 'input': 'object'}
COLUMNS = ['block_number', 'block_timestamp', 'hash', 'from_address', 'to_address', 'value', 'gas', 'gas_price', 'receipt_status', 'receipt_contract_address','input']

train_dfs = dd.read_csv(f'{data_dir}/eoa_tx_train/eoa_tx_train/*.csv', dtype=TX_DTYPES, usecols=COLUMNS)
test_dfs = dd.read_csv(f'{data_dir}/eoa_tx_test/eoa_tx_test/*.csv', dtype=TX_DTYPES, usecols=COLUMNS)

train_dfs = train_dfs.rename(columns={
    'block_timestamp': 'timestamp',
    'hash': 'transaction_hash'
})
test_dfs = test_dfs.rename(columns={
    'block_timestamp': 'timestamp',
    'hash': 'transaction_hash'
})

In [5]:
# tracing

data_dir = 'kaggle_data/forta-protect-web3'

TX_DTYPES = {'value': 'object', 'status': 'object', 'input': 'object'}
COLUMNS = ['block_number', 'block_timestamp', 'transaction_hash', 'from_address', 'to_address', 'value', 'trace_type','status','input']

train_internal_dfs = dd.read_csv(f'{data_dir}/tx_trace_train/tx_trace_train/*.csv', dtype=TX_DTYPES, usecols=COLUMNS)
test_internal_dfs = dd.read_csv(f'{data_dir}/tx_trace_test/tx_trace_test/*.csv', dtype=TX_DTYPES, usecols=COLUMNS)

train_internal_dfs = train_internal_dfs.rename(columns={
    'block_timestamp': 'timestamp'
})
test_internal_dfs = test_internal_dfs.rename(columns={
    'block_timestamp': 'timestamp'
})

In [6]:
# token transaction

data_dir = 'kaggle_data/forta-protect-web3'

TX_DTYPES = {'value': 'object'}
COLUMNS = ['blockNumber', 'timestamp', 'transactionHash', 'from_addr', 'to_addr', 'value', 'tokenID', 'tokenAddress', 'edge_type']

train_token_dfs = dd.read_csv(f'{data_dir}/tx_token_train/*.csv', dtype=TX_DTYPES, usecols=COLUMNS)
test_token_dfs = dd.read_csv(f'{data_dir}/tx_token_test/*.csv', dtype=TX_DTYPES, usecols=COLUMNS)

train_token_dfs = train_token_dfs.rename(columns={
    'blockNumber': 'block_number', 'transactionHash': 'transaction_hash',
    'from_addr': 'from_address', 'to_addr': 'to_address', 'tokenID': 'token_id', 'tokenAddress': 'token_address'
})

test_token_dfs = test_token_dfs.rename(columns={
    'blockNumber': 'block_number', 'transactionHash': 'transaction_hash',
    'from_addr': 'from_address', 'to_addr': 'to_address', 'tokenID': 'token_id', 'tokenAddress': 'token_address'
})

In [7]:
# Get training ground truth values and ids

COLUMNS = ['address']
train_ids = pd.read_csv(f'{data_dir}/train.csv', usecols=COLUMNS)
test_ids = pd.read_csv(f'{data_dir}/test.csv', usecols=COLUMNS)

In [8]:
# filter out the unsuccessful transactions

unsuccessful_trainsaction_hash = set(list(train_dfs[train_dfs['receipt_status'] != '1']['transaction_hash'].compute())) | set(list(test_dfs[test_dfs['receipt_status'] != '1']['transaction_hash'].compute()))

train_dfs = train_dfs[(train_dfs['receipt_status'] == '1') & (train_dfs['timestamp'].notnull())]  # 还要过滤区块时间为空的
test_dfs = test_dfs[(test_dfs['receipt_status'] == '1') & (test_dfs['timestamp'].notnull())]

train_internal_dfs = train_internal_dfs[(train_internal_dfs['status'] == '1') & (train_internal_dfs['timestamp'].notnull())]
test_internal_dfs = test_internal_dfs[(test_internal_dfs['status'] == '1') & (test_internal_dfs['timestamp'].notnull())]

train_token_dfs = train_token_dfs[~train_token_dfs['transaction_hash'].isin(unsuccessful_trainsaction_hash)]
test_token_dfs = test_token_dfs[~test_token_dfs['transaction_hash'].isin(unsuccessful_trainsaction_hash)]

In [9]:
# EOA transaction

incoming = train_dfs.merge(train_ids, how='left', left_on='to_address', right_on='address')
incoming = incoming[incoming['address'].notnull()].compute()

outgoing = train_dfs.merge(train_ids, how='left', left_on='from_address', right_on='address')
outgoing = outgoing[outgoing['address'].notnull()].compute()

contract_creation = train_dfs.merge(train_ids, how='left', left_on='receipt_contract_address', right_on='address')
contract_creation = contract_creation[contract_creation['address'].notnull()].compute()

train_dfs = pd.concat([incoming, outgoing, contract_creation]).reset_index(drop=True)

del incoming, outgoing, contract_creation

'''test_dfs   test_ids'''
incoming = test_dfs.merge(test_ids, how='left', left_on='to_address', right_on='address')
incoming = incoming[incoming['address'].notnull()].compute()

outgoing = test_dfs.merge(test_ids, how='left', left_on='from_address', right_on='address')
outgoing = outgoing[outgoing['address'].notnull()].compute()

contract_creation = test_dfs.merge(test_ids, how='left', left_on='receipt_contract_address', right_on='address')
contract_creation = contract_creation[contract_creation['address'].notnull()].compute()

test_dfs = pd.concat([incoming, outgoing, contract_creation]).reset_index(drop=True)

del incoming, outgoing, contract_creation

In [10]:
#EOA transaction

train_dfs.loc[train_dfs[(train_dfs['value'] == '0') & (train_dfs['to_address'].notnull())].index.tolist(), 'edge_type'] = 'contract_call'
train_dfs.loc[train_dfs[train_dfs['receipt_contract_address'].notnull()].index.tolist(), 'edge_type'] = 'contract_create'
train_dfs.loc[train_dfs[train_dfs['receipt_contract_address'].notnull()].index.tolist(), 'to_address'] = train_dfs['receipt_contract_address']
train_dfs.loc[train_dfs[(train_dfs['value'] != '0') & (train_dfs['input'] != '0x') & (train_dfs['input'] != '0x00')].index.tolist(), 'edge_type'] = 'call_and_transfer'
train_dfs.loc[train_dfs[(train_dfs['value'] != '0') & ((train_dfs['input'] == '0x') | (train_dfs['input'] == '0x00'))].index.tolist(), 'edge_type'] = 'external_transfer'
train_dfs = train_dfs.drop(['receipt_contract_address','input', 'receipt_status'], axis=1)


'''test_dfs   test_ids'''
test_dfs.loc[test_dfs[(test_dfs['value'] == '0') & (test_dfs['to_address'].notnull())].index.tolist(), 'edge_type'] = 'contract_call'
test_dfs.loc[test_dfs[test_dfs['receipt_contract_address'].notnull()].index.tolist(), 'edge_type'] = 'contract_create'
test_dfs.loc[test_dfs[test_dfs['receipt_contract_address'].notnull()].index.tolist(), 'to_address'] = test_dfs['receipt_contract_address']
test_dfs.loc[test_dfs[(test_dfs['value'] != '0') & (test_dfs['input'] != '0x') & (test_dfs['input'] != '0x00')].index.tolist(), 'edge_type'] = 'call_and_transfer'
test_dfs.loc[test_dfs[(test_dfs['value'] != '0') & ((test_dfs['input'] == '0x') | (test_dfs['input'] == '0x00'))].index.tolist(), 'edge_type'] = 'external_transfer'
test_dfs = test_dfs.drop(['receipt_contract_address','input', 'receipt_status'], axis=1)

In [11]:
# tracing data

incoming = train_internal_dfs.merge(train_ids, how='left', left_on='to_address', right_on='address')
incoming = incoming[incoming['address'].notnull()].compute()

outgoing = train_internal_dfs.merge(train_ids, how='left', left_on='from_address', right_on='address')
outgoing = outgoing[outgoing['address'].notnull()].compute()

train_internal_dfs = pd.concat([incoming, outgoing]).reset_index(drop=True)

del incoming, outgoing

'''test_internal_dfs   test_ids'''

incoming = test_internal_dfs.merge(test_ids, how='left', left_on='to_address', right_on='address')
incoming = incoming[incoming['address'].notnull()].compute()

outgoing = test_internal_dfs.merge(test_ids, how='left', left_on='from_address', right_on='address')
outgoing = outgoing[outgoing['address'].notnull()].compute()

test_internal_dfs = pd.concat([incoming, outgoing]).reset_index(drop=True)

del incoming, outgoing

In [25]:
# tracing process
train_internal_dfs.loc[train_internal_dfs[train_internal_dfs['trace_type'] == 'reward'].index.tolist(), 'edge_type'] = 'mining_reward'
train_internal_dfs.loc[train_internal_dfs[train_internal_dfs['trace_type'] == 'genesis'].index.tolist(), 'edge_type'] = 'mining_reward'
train_internal_dfs.loc[train_internal_dfs[train_internal_dfs['trace_type'] == 'suicide'].index.tolist(), 'edge_type'] = 'suicide'
train_internal_dfs.loc[train_internal_dfs[(train_internal_dfs['value'] != '0') & (train_internal_dfs['trace_type'] == 'suicide')].index.tolist(), 'edge_type'] = 'suicide and transfer'
train_internal_dfs.loc[train_internal_dfs[train_internal_dfs['trace_type'] == 'create'].index.tolist(), 'edge_type'] = 'contract_create'
train_internal_dfs.loc[train_internal_dfs[(train_internal_dfs['value'] != '0') & ((train_internal_dfs['input'] == '') | (train_internal_dfs['input'] == '0x') | (train_internal_dfs['input'] == '0x00')) & (train_internal_dfs['trace_type'] == 'call')].index.tolist(), 'edge_type'] = 'internal_transfer'
train_internal_dfs.loc[train_internal_dfs[(train_internal_dfs['value'] != '0') & (train_internal_dfs['input'] != '') & (train_internal_dfs['input'] != '0x') & (train_internal_dfs['input'] != '0x00') & (train_internal_dfs['trace_type'] == 'call')].index.tolist(), 'edge_type'] = 'call_and_internal_transfer'
train_internal_dfs.loc[train_internal_dfs[(train_internal_dfs['value'] == '0') & (train_internal_dfs['trace_type'] == 'call')].index.tolist(), 'edge_type'] = 'contract_call'
train_internal_dfs = train_internal_dfs.drop(['trace_type','status','input'], axis=1)


'''test_internal_dfs   test_ids'''
test_internal_dfs.loc[test_internal_dfs[test_internal_dfs['trace_type'] == 'reward'].index.tolist(), 'edge_type'] = 'mining_reward'
test_internal_dfs.loc[test_internal_dfs[test_internal_dfs['trace_type'] == 'genesis'].index.tolist(), 'edge_type'] = 'mining_reward'
test_internal_dfs.loc[test_internal_dfs[test_internal_dfs['trace_type'] == 'suicide'].index.tolist(), 'edge_type'] = 'suicide'
test_internal_dfs.loc[test_internal_dfs[(test_internal_dfs['value'] != '0') & (test_internal_dfs['trace_type'] == 'suicide')].index.tolist(), 'edge_type'] = 'suicide and transfer'
test_internal_dfs.loc[test_internal_dfs[test_internal_dfs['trace_type'] == 'create'].index.tolist(), 'edge_type'] = 'contract_create'
test_internal_dfs.loc[test_internal_dfs[(test_internal_dfs['value'] != '0') & ((test_internal_dfs['input'] == '') | (test_internal_dfs['input'] == '0x') | (test_internal_dfs['input'] == '0x00')) & (test_internal_dfs['trace_type'] == 'call')].index.tolist(), 'edge_type'] = 'internal_transfer'
test_internal_dfs.loc[test_internal_dfs[(test_internal_dfs['value'] != '0') & (test_internal_dfs['input'] != '') & (test_internal_dfs['input'] != '0x') & (test_internal_dfs['input'] != '0x00') & (test_internal_dfs['trace_type'] == 'call')].index.tolist(), 'edge_type'] = 'call_and_internal_transfer'
test_internal_dfs.loc[test_internal_dfs[(test_internal_dfs['value'] == '0') & (test_internal_dfs['trace_type'] == 'call')].index.tolist(), 'edge_type'] = 'contract_call'
test_internal_dfs = test_internal_dfs.drop(['trace_type','status','input'], axis=1)


In [24]:
# token
incoming = train_token_dfs.merge(train_ids, how='left', left_on='to_address', right_on='address')
incoming = incoming[incoming['address'].notnull()].compute()

outgoing = train_token_dfs.merge(train_ids, how='left', left_on='from_address', right_on='address')
outgoing = outgoing[outgoing['address'].notnull()].compute()

train_token_dfs = pd.concat([incoming, outgoing]).reset_index(drop=True)

del incoming, outgoing

'''test_token_dfs  test_ids'''
incoming = test_token_dfs.merge(test_ids, how='left', left_on='to_address', right_on='address')
incoming = incoming[incoming['address'].notnull()].compute()

outgoing = test_token_dfs.merge(test_ids, how='left', left_on='from_address', right_on='address')
outgoing = outgoing[outgoing['address'].notnull()].compute()

test_token_dfs = pd.concat([incoming, outgoing]).reset_index(drop=True)

del incoming, outgoing

/media/ubuntu/ElementsSE/2022_finvcup_baseline-master/env/bisai/lib/python3.7/site-packages/distributed/worker.py:4504: UserWarning: Large object of size 138.48 MiB detected in task graph: 
  (['0x5635158fc766e63b060873503a27709b50ee39048c090 ... 4f9976ac45d'],)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s)
distributed.utils_perf - WARNING - full garbage collections took 46% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 47% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 48% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
distribute

In [26]:
print(train_dfs.columns)

print(train_internal_dfs.columns)

print(train_token_dfs.columns)

print(len(train_dfs))

print(len(train_internal_dfs))

print(len(train_token_dfs))

print(test_dfs.columns)

print(test_internal_dfs.columns)

print(test_token_dfs.columns)

print(len(test_dfs))

print(len(test_internal_dfs))

print(len(test_token_dfs))



Index(['transaction_hash', 'from_address', 'to_address', 'value', 'gas',
       'gas_price', 'timestamp', 'block_number', 'address', 'edge_type'],
      dtype='object')
Index(['transaction_hash', 'from_address', 'to_address', 'value', 'timestamp',
       'block_number', 'address', 'edge_type'],
      dtype='object')
Index(['block_number', 'timestamp', 'transaction_hash', 'from_address',
       'to_address', 'edge_type', 'value', 'token_id', 'token_address',
       'address'],
      dtype='object')
15644709
17695161
586
Index(['transaction_hash', 'from_address', 'to_address', 'value', 'gas',
       'gas_price', 'timestamp', 'block_number', 'address', 'edge_type'],
      dtype='object')
Index(['transaction_hash', 'from_address', 'to_address', 'value', 'timestamp',
       'block_number', 'address', 'edge_type'],
      dtype='object')
Index(['block_number', 'timestamp', 'transaction_hash', 'from_address',
       'to_address', 'edge_type', 'value', 'token_id', 'token_address',
       'addre

In [27]:
# concat the data
train_tx = pd.concat([train_dfs, train_internal_dfs, train_token_dfs]).reset_index(drop=True)
test_tx = pd.concat([test_dfs, test_internal_dfs, test_token_dfs]).reset_index(drop=True)

del train_dfs, train_internal_dfs, train_token_dfs, test_dfs, test_internal_dfs, test_token_dfs

In [28]:
train_tx.to_csv('train_tx.csv', index=None)
test_tx.to_csv('test_tx.csv', index=None)